In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_flipkart_smartphones(search_query):
    base_url = f"https://www.flipkart.com/search?q={search_query.replace(' ', '%20')}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
    }
    response = requests.get(base_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    results = []
    for card in soup.find_all('div', class_='_1AtVbE'):
        try:
            product_url = 'https://www.flipkart.com' + card.find('a', class_='IRpwTa')['href']
        except:
            product_url = '-'

        try:
            title = card.find('div', class_='_4rR01T').text.strip()
        except:
            title = '-'

        try:
            brand_name = title.split(' ')[0]
        except:
            brand_name = '-'

        try:
            smartphone_name = ' '.join(title.split(' ')[1:])
        except:
            smartphone_name = '-'

        try:
            colour = card.find('a', class_='IRpwTa')['title']
        except:
            colour = '-'

        try:
            details = card.find_all('li', class_='rgWa7D')
            ram = '-'
            rom = '-'
            primary_camera = '-'
            secondary_camera = '-'
            display_size = '-'
            battery_capacity = '-'
            for detail in details:
                if 'RAM' in detail.text:
                    ram = detail.text.split('|')[0].strip()
                elif 'ROM' in detail.text:
                    rom = detail.text.split('|')[0].strip()
                elif 'Primary Camera' in detail.text:
                    primary_camera = detail.text.split('|')[0].strip()
                elif 'Secondary Camera' in detail.text:
                    secondary_camera = detail.text.split('|')[0].strip()
                elif 'Display Size' in detail.text:
                    display_size = detail.text.split('|')[0].strip()
                elif 'Battery Capacity' in detail.text:
                    battery_capacity = detail.text.split('|')[0].strip()
        except:
            pass

        try:
            price = card.find('div', class_='_30jeq3 _1_WHN1').text.strip()
        except:
            price = '-'

        results.append({
            'Brand Name': brand_name,
            'Smartphone Name': smartphone_name,
            'Colour': colour,
            'RAM': ram,
            'Storage(ROM)': rom,
            'Primary Camera': primary_camera,
            'Secondary Camera': secondary_camera,
            'Display Size': display_size,
            'Battery Capacity': battery_capacity,
            'Price': price,
            'Product URL': product_url
        })

    return results

# Search query for smartphones
search_query = "Oneplus Nord"

# Scraping data
results = scrape_flipkart_smartphones(search_query)

# Creating DataFrame
df = pd.DataFrame(results)

# Saving to CSV
df.to_csv('flipkart_smartphones.csv', index=False)
